# USGS API
My own "tutorial".

The idea is to send url queries to the usgs db

In [1]:
from lxml import html
import requests

First I study the GeoJson format:

http://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

In [2]:
# s = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_week.geojson"
s = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
# s = "http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02"
r = requests.get(s)

Convert the json string to a dictionary

In [3]:
import json
d = json.loads(r.content)
d.keys()

[u'type', u'features', u'bbox', u'metadata']

As is shown in the website address mentioned above, each file is of this form:

```json
{
  type: "FeatureCollection",
  metadata: { 
  ...
  },
  bbox: [
    ...
  ],
  features: [
    {
      type: "Feature",
      properties: {
        mag: Decimal,
        ...
        tsunami: Integer,
        ...
        magType: String,
        type: String
      },
      geometry: {
        type: "Point",
        coordinates: [
          longitude,
          latitude,
          depth
        ]
      },
      id: String
    },
    …
  ]
}
```

In [4]:
for f in d['features']:
    coords = f['geometry']['coordinates']
    place = f['properties']['place']
    mag = f['properties']['mag']
    magType = f['properties']['magType'] 
    tsunamiFlag = f['properties']['tsunami']
    Type = f['properties']['type']
    print 'place = %s \n\tcoords ='%place,coords
    print '\tmag = %.3f%sm tsunami = %i Type = %s'%(mag,magType,tsunamiFlag, Type)

place = 3km ESE of Piedmont, California 
	coords = [-122.1965027, 37.8074989, 4.75]
	mag = 3.540mwm tsunami = 0 Type = earthquake
place = 8km S of Kyonju, South Korea 
	coords = [129.2267, 35.7686, 10]
	mag = 5.400mbm tsunami = 0 Type = earthquake
place = 4km ENE of Skopje, Macedonia 
	coords = [21.4809, 42.0124, 10]
	mag = 5.300mbm tsunami = 0 Type = earthquake
place = 22km NE of Nsunga, Tanzania 
	coords = [31.5604, -1.0302, 40]
	mag = 5.900mwwm tsunami = 0 Type = earthquake
place = 0km WSW of Spencer, Oklahoma 
	coords = [-97.3848, 35.5213, 5]
	mag = 3.800mb_lgm tsunami = 0 Type = earthquake
place = 19km ENE of Sungjibaegam, North Korea 
	coords = [129.0147, 41.2977, 0]
	mag = 5.300mbm tsunami = 0 Type = explosion
place = 189km ESE of Nikol'skoye, Russia 
	coords = [168.5734, 54.3674, 12.32]
	mag = 6.300mwpm tsunami = 0 Type = earthquake
place = 15km NW of Pawnee, Oklahoma 
	coords = [-96.9317, 36.4298, 5.4]
	mag = 5.800mwwm tsunami = 0 Type = earthquake
place = 103km W of Ferndale,

# Sending Queries to the USGS API

The API can give many kind of answers (methods), as is well explained here

http://earthquake.usgs.gov/fdsnws/event/1/

The first kind of interest is the "count" method which indicates how many results are associated to the querystring

http://earthquake.usgs.gov/fdsnws/event/1/count?format=geojson&starttime=2014-01-01&endtime=2014-01-02

The other is the "query" method, which actually performs the query

http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02

Two kind of queries that can be interesting are:
1. Get all events bigger than a certain magnitude, considering the whole planet.
2. Given a point location, return all events "close" to that point, or the closest one. Close means a predefined radius.

For now, only the first kind of query.

## Query all events with magnitude over 9.0

In [5]:
baseCountString = "http://earthquake.usgs.gov/fdsnws/event/1/count?format=geojson"
baseQueryString = "http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
startTime = "&starttime=0000-01-01"
endTime = "&endtime=2016-01-02"
minMagnitudeString = "&minmagnitude=9.0"
qString = baseQueryString + startTime + endTime + minMagnitudeString
cString = baseCountString + startTime + endTime + minMagnitudeString

In [6]:
countMag9 = requests.get(cString)
dCountMag9 = json.loads(countMag9.content)
dCountMag9

{u'count': 4, u'maxAllowed': 20000}

In [7]:
queryMag9 = requests.get(qString)
dQueryMag9 = json.loads(queryMag9.content)

In [8]:
for f in dQueryMag9['features']:
    coords = f['geometry']['coordinates']
    place = f['properties']['place']
    mag = f['properties']['mag']
    magType = f['properties']['magType'] 
    tsunamiFlag = f['properties']['tsunami']
    Type = f['properties']['type']
    print 'place = %s \n\tcoords ='%place,coords
    print '\tmag = %.3f%sm tsunami = %i Type = %s'%(mag,magType,tsunamiFlag, Type)

place = near the east coast of Honshu, Japan 
	coords = [142.373, 38.297, 29]
	mag = 9.000mwwm tsunami = 0 Type = earthquake
place = off the west coast of northern Sumatra 
	coords = [95.982, 3.295, 30]
	mag = 9.100mwm tsunami = 0 Type = earthquake
place = Southern Alaska 
	coords = [-147.339, 60.908, 25]
	mag = 9.300mwm tsunami = 0 Type = earthquake
place = Bio-Bio, Chile 
	coords = [-73.407, -38.143, 25]
	mag = 9.600mwm tsunami = 0 Type = earthquake


In [9]:
f["properties"].keys()

[u'rms',
 u'code',
 u'cdi',
 u'sources',
 u'nst',
 u'tz',
 u'title',
 u'magType',
 u'detail',
 u'sig',
 u'net',
 u'type',
 u'status',
 u'updated',
 u'felt',
 u'alert',
 u'dmin',
 u'mag',
 u'gap',
 u'types',
 u'url',
 u'ids',
 u'tsunami',
 u'place',
 u'time',
 u'mmi']

# Moment tensor query

In [10]:
baseQueryString = "http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
startTime = "&starttime=0000-01-01"
endTime = "&endtime=2016-01-02"
minMagnitudeString = "&minmagnitude=9.0"
productType = "&producttype=moment-tensor"
qString = baseQueryString + startTime + endTime + minMagnitudeString + productType

In [11]:
print  qString

http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=0000-01-01&endtime=2016-01-02&minmagnitude=9.0&producttype=moment-tensor


In [12]:
queryMag9_withMoment = requests.get(qString)
dQueryWithMoment = json.loads(queryMag9_withMoment.content)

In [13]:
for f in dQueryWithMoment['features']:
    coords = f['geometry']['coordinates']
    place = f['properties']['place']
    mag = f['properties']['mag']
    magType = f['properties']['magType'] 
    tsunamiFlag = f['properties']['tsunami']
    Type = f['properties']['type']
    print 'place = %s \n\tcoords ='%place,coords
    print '\tmag = %.3f%sm tsunami = %i Type = %s'%(mag,magType,tsunamiFlag, Type)

place = near the east coast of Honshu, Japan 
	coords = [142.373, 38.297, 29]
	mag = 9.000mwwm tsunami = 0 Type = earthquake
place = off the west coast of northern Sumatra 
	coords = [95.982, 3.295, 30]
	mag = 9.100mwm tsunami = 0 Type = earthquake


Query the moment-tensor data for a particular scenario

In [22]:
f["id"]
eventIdString ="&eventid=%s"%f["id"]
qMomentString = baseQueryString + eventIdString
print qMomentString

http://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&eventid=official20041226005853450_30


In [23]:
queryMoment = requests.get(qMomentString)
dQueryMoment = json.loads(queryMoment.content)

In [28]:
dQueryMoment

{u'geometry': {u'coordinates': [95.982, 3.295, 30], u'type': u'Point'},
 u'id': u'official20041226005853450_30',
 u'properties': {u'alert': None,
  u'cdi': 9.1,
  u'code': u'20041226005853450_30',
  u'dmin': None,
  u'felt': 886,
  u'gap': 22,
  u'ids': u',duputel122604a,us2004slav,official20041226005853450_30,choy20041226005853,usp000dbed,gcmt20041226005850,atlas20041226005853,iscgem7453151,',
  u'mag': 9.1,
  u'magType': u'mw',
  u'mmi': 8.49,
  u'net': u'official',
  u'nst': 601,
  u'place': u'off the west coast of northern Sumatra',
  u'products': {u'associate': [{u'code': u'official20041226005853450_30_duputel122604a',
     u'contents': {u'': {u'bytes': u'outside automatic association window',
       u'contentType': u'1',
       u'lastModified': 1455831310000,
       u'length': 36}},
     u'id': u'urn:usgs-product:admin:associate:official20041226005853450_30_duputel122604a:1455831310225',
     u'indexTime': 1455831311005,
     u'indexid': u'80419421',
     u'preferredWeight': 1,
 

In [49]:
tensors = dQueryMoment['properties']['products']['moment-tensor']
for i in range(len((tensors))):
    m = dQueryMoment['properties']['products']['moment-tensor'][i]
    print 'w=%s, dip=%s, rake=%s, strike=%s'%(m['preferredWeight'], m['properties']['nodal-plane-1-dip'], \
            m['properties']['nodal-plane-1-rake'], m['properties']['nodal-plane-1-strike'])

w=161, dip=7, rake=113, strike=335
w=159, dip=8, rake=110, strike=329
w=6, dip=8, rake=110, strike=329
w=6, dip=13, rake=55, strike=274
